In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
from shapely.ops import transform

In [9]:
def prep_reach_dist(wel_lon, wel_lat, stream_sf, reach_id, stream_pt_spacing, nseed=1):
        """
    Calculate the distance from a well to each reach within a stream network.
    This function splits a polyline stream network into evenly spaced points and calculates the distance
    from each of those points to a well.

    Parameters:
        wel_lon (float): Longitude of the well.
        wel_lat (float): Latitude of the well.
        stream_sf (GeoDataFrame): GeoDataFrame containing stream lines.
        reach_id (str): Column name in `stream_sf` for the unique identifier of each stream line segment.
        stream_pt_spacing (float): Distance between points used for sampling each stream reach.
        nseed (int): Seed for random number generator (used for point sampling).

    Returns:
        DataFrame: A DataFrame with columns ['reach', 'dist', 'lat', 'lon'].
    """
    np.random.seed(nseed)
    
    # Helper function to sample points or create midpoint for short geometries
    def sample_stream_points(geometry, spacing):
        if geometry.is_empty or not isinstance(geometry, LineString):
            return []  # Skip invalid or empty geometries
        length = geometry.length
        if length < spacing:  # For short lines, add midpoint
            return [geometry.interpolate(0.5 * length)]
        num_points = int(np.ceil(length / spacing))
        return [geometry.interpolate(i * spacing) for i in range(num_points)]
    
    # Generate points and ensure all ReachCodes are included
    points = []
    for _, row in stream_sf.iterrows():
        reach_points = sample_stream_points(row.geometry, stream_pt_spacing)
        for point in reach_points:
            points.append({
                'reach': row[reach_id],
                'lon': point.x,
                'lat': point.y
            })
        # Ensure at least one record for every ReachCode, even if no points are generated
        if not reach_points:
            points.append({
                'reach': row[reach_id],
                'lon': row.geometry.centroid.x,
                'lat': row.geometry.centroid.y
            })
    
    # Convert to GeoDataFrame and calculate distances
    points_gdf = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy([p['lon'] for p in points], [p['lat'] for p in points]))
    points_gdf['dist'] = np.sqrt((points_gdf['lon'] - wel_lon)**2 + (points_gdf['lat'] - wel_lat)**2)
    
    return points_gdf[['reach', 'dist', 'lat', 'lon']]

In [11]:
stream_sf = gpd.read_file('C:/Users/betebari/Documents/_Jupyter/StrmDepletr/shapefiles/MWC_NHD_flowlines_utm.shp')

In [21]:
result = prep_reach_dist(
    wel_lon=522984,
    wel_lat=4270707,
    stream_sf=stream_sf,
    reach_id='ReachCode', 
    stream_pt_spacing=5
)

# Verify all ReachCodes are in the result
print(result['reach'].nunique(), "out of", stream_sf['ReachCode'].nunique())
print(result.head())

1021 out of 1021
            reach          dist           lat            lon
0  18010110002256  16046.044883  4.262655e+06  536863.269445
1  18010110002256  16045.388187  4.262659e+06  536865.188138
2  18010110002256  16044.733023  4.262664e+06  536867.106830
3  18010110002256  16044.079391  4.262668e+06  536869.025522
4  18010110002256  16043.427289  4.262673e+06  536870.944215


In [15]:
print(stream_sf['ReachCode'].head())

0    18010110002256
1    18010110003814
2    18010110001108
3    18010110001336
4    18010110000050
Name: ReachCode, dtype: object


In [17]:
print(stream_sf.geom_type.unique())

['LineString']


In [23]:
# Display the unique ReachCodes present in the result
print(result['reach'].unique())

# Check how many rows belong to each ReachCode
print(result['reach'].value_counts())

# Filter results for a specific ReachCode (e.g., 18010110003814)
print(result[result['reach'] == '18010110003814'].head())

['18010110002256' '18010110003814' '18010110001108' ... '18050002006234'
 '18050002007346' '18050002000217']
reach
18010110000463    2672
18010110000070    1800
18010110000035    1747
18010110001254    1631
18010110000468    1578
                  ... 
18010110004712       5
18050002006327       3
18050002008325       3
18010110010891       2
18010110001205       1
Name: count, Length: 1021, dtype: int64
             reach         dist           lat            lon
57  18010110003814  2963.900973  4.271294e+06  525889.124096
58  18010110003814  2962.024343  4.271299e+06  525886.363042
59  18010110003814  2960.154969  4.271303e+06  525883.601988
60  18010110003814  2958.292865  4.271307e+06  525880.840934
61  18010110003814  2956.438044  4.271311e+06  525878.079879


In [31]:
result_sorted = result.sort_values(by='reach')
print(result_sorted.head(100))

                reach          dist           lat            lon
34025  18010110000020  16284.443624  4.260519e+06  510280.087791
73172  18010110000020  14660.820938  4.260390e+06  512568.065562
73173  18010110000020  14665.813249  4.260386e+06  512564.713821
73174  18010110000020  14670.805566  4.260382e+06  512561.362080
73175  18010110000020  14675.797887  4.260379e+06  512558.010339
...               ...           ...           ...            ...
73091  18010110000020  14271.614419  4.260660e+06  512848.468019
73083  18010110000020  14232.304880  4.260688e+06  512875.434685
73092  18010110000020  14276.134593  4.260658e+06  512843.889436
73094  18010110000020  14283.839474  4.260657e+06  512833.929444
73095  18010110000020  14287.692982  4.260656e+06  512828.949448

[100 rows x 4 columns]


In [27]:
result.to_csv('stream_distances.csv', index=False)

In [29]:
grouped = result.groupby('reach').size()
print(grouped)

reach
18010110000020    990
18010110000021    715
18010110000022     11
18010110000032    526
18010110000033    471
                 ... 
18050002008362    107
18050002008364    170
18050002008365    115
18050002008379     21
18050002008472     95
Length: 1021, dtype: int64
